In [1]:
!pip install transformers
!pip install peft
!pip install 'accelerate>=0.26.0'
!pip install -U bitsandbytes
!pip install huggingface-hub

In [3]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
from torch.utils.data import Dataset
import transformers
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
from peft import LoraConfig, get_peft_model
import argparse
from huggingface_hub import login
import os
import logging


class CustomTextDataset(Dataset):
    def __init__(self, texts, tokenizer):
        self.texts = texts
        self.tokenizer = tokenizer

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        text = str(self.texts[idx])

        encoding = self.tokenizer(
            text,
            return_tensors='pt'
        )

        return {
            'input_ids': encoding['input_ids'].flatten(),
            'attention_mask': encoding['attention_mask'].flatten(),
        }


def create_prompted_text(
    dataset,
    label_name
):
    texts = []
    for _, row in dataset.iterrows():
        text = """Based on a part of the interview where the interviewer asks a
        set of questions, classify the type of answer the interviewee provided
        for the following question.\n\n ### Part of the interview ###\n """
        text += row["interview_question"] + "\n" + row["interview_answer"]
        text += "\n\n" + "### Question ###\n\n"
        text += row["question"] + "\nLabel: " + row[label_name] + "\n\n"
        texts.append(text)

    return texts


def load_qevasion_dataset(
    tokenizer,
    label_name="clarity_label"
):
    # Get train set data
    df = pd.read_csv('preprocessed_data/train_set.csv')[['question',
                                                         'interview_question',
                                                         'interview_answer',
                                                         'clarity_label']]

    # Split train set to train and validation data
    np.random.seed(2024)
    msk = np.random.rand(len(df)) < 0.8
    train = df[msk]
    validation = df[~msk]

    train.reset_index(drop=True, inplace=True)
    validation.reset_index(drop=True, inplace=True)

    train_texts = create_prompted_text(train, label_name)
    validation_texts = create_prompted_text(validation,
                                            label_name)

    print("Example of train test:" + train_texts[1])
    print("Example of validation test:" + validation_texts[1])

    return (CustomTextDataset(train_texts, tokenizer),
            CustomTextDataset(validation_texts, tokenizer))


class CastOutputToFloat(nn.Sequential):
    def forward(self, x): return super().forward(x).to(torch.float32)


def print_trainable_parameters(model):
    """
    Prints the number of trainable parameters in the model.
    """
    trainable_params = 0
    all_param = 0
    for _, param in model.named_parameters():
        all_param += param.numel()
        if param.requires_grad:
            trainable_params += param.numel()
    print(
        f"""trainable params: {trainable_params} || all params: {all_param}
        || trainable%: {100 * trainable_params / all_param}"""
    )


def main(model_name,
         output_model_dir,
         label_taxonomy):

    cache_dir = ""

    model = AutoModelForCausalLM.from_pretrained(
        model_name,
        # load_in_4bit=True,
        quantization_config=BitsAndBytesConfig(
            load_in_4bit=True,
            bnb_4bit_compute_dtype=torch.bfloat16
        ),
        device_map='auto',
        torch_dtype=torch.float16,
        cache_dir=cache_dir
    )

    tokenizer = AutoTokenizer.from_pretrained(model_name, cache_dir=cache_dir)
    tokenizer.add_special_tokens({'pad_token': '[PAD]'})

    tokenizer = AutoTokenizer.from_pretrained(model_name, cache_dir=cache_dir,
                                              trust_remote_code=True,)
    tokenizer.add_special_tokens({'pad_token': '[PAD]'})

    for param in model.parameters():
        param.requires_grad = False  # freeze the model - train adapters later
        if param.ndim == 1:
            # cast the small parameters (e.g. layernorm) to fp32 for stability
            param.data = param.data.to(torch.float32)

    # Reduce number of stored activation
    model.gradient_checkpointing_enable()
    model.enable_input_require_grads()
    model.lm_head = CastOutputToFloat(model.lm_head)

    config = LoraConfig(
        r=16,  # Attention heads
        lora_alpha=32,  # Alpha scaling
        # target_modules=["q_proj", "v_proj"],
        lora_dropout=0.05,
        bias="none",
        task_type="CAUSAL_LM"  # set this for CLM or Seq2Seq
    )

    model = get_peft_model(model, config)
    print_trainable_parameters(model)

    # load data
    train_data, validation_data = load_qevasion_dataset(tokenizer,
                                                        label_taxonomy)

    print(f"""Found {len(train_data)} instances for training and
    {len(validation_data) } instances for validation.""")

    epochs = 1

    # Train model
    print("Training...")
    out_dir = output_model_dir.split("/")[-1]
    trainer = transformers.Trainer(
        model=model,
        train_dataset=train_data,
        eval_dataset=validation_data,
        args=transformers.TrainingArguments(
            per_device_train_batch_size=1,
            gradient_accumulation_steps=1,
            warmup_steps=100,
            max_steps=len(train_data) * epochs,
            learning_rate=2e-4,
            fp16=True,
            logging_steps=10,
            eval_steps=10,
            # eval_strategy="steps",
            do_eval=True,
            output_dir=f'outputs_{out_dir}'
        ),
        data_collator=transformers.DataCollatorForLanguageModeling(tokenizer,
                                                                   mlm=False)
    )
    # silence the warnings. Re-enable for inference!
    model.config.use_cache = False
    trainer.train()
    # Save the model
    model.save_pretrained(output_model_dir)

    # Optionally, save the tokenizer as well
    tokenizer.save_pretrained(output_model_dir)

In [4]:
login(os.environ["hf_key"])

main('meta-llama/Llama-3.1-8B-Instruct',
     output_model_dir="./data",
     label_taxonomy="clarity_label")

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

trainable params: 6815744 || all params: 4547416064
        || trainable%: 0.14988168894325302
Example of train test:Based on a part of the interview where the interviewer asks a
        set of questions, classify the type of answer the interviewee provided
        for the following question.

 ### Part of the interview ###
 Q. Of the Biden administration. And accused the United States of containing China while pushing for diplomatic talks.How would you respond to that? And do you think President Xi is being sincere about getting the relationship back on track as he bans Apple in China?
Well, look, first of all, theI am sincere about getting the relationship right. And one of the things that is going on now is, China is beginning to change some of the rules of the game, in terms of trade and other issues.And so one of the things we talked about, for example, is that they're now talking about making sure that no Chineseno one in the Chinese Government can use a Western cell phone. Those

Step,Training Loss
10,2.879800
20,2.898600
30,2.676400
40,2.503200


KeyboardInterrupt: 